<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
📏 תהליך הסבת שכבת חזיתות לבנק"ל מודרני 📏 

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
print('9. Fronts:')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import subprocess
import numpy as np
import pandas as pd
import geopandas as gpd

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  9.a Reading inputs')

In [ ]:
RawData_Folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

Results_Gdb = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\Results.gdb'

<h1 style="text-align:right;font-size:125%">
שכבת חזיתות

In [ ]:
arc_cols = ['LPOLY_', 'RPOLY_', 'RADIUS', 'FNODE_', 'TNODE_', 'Shape_Length', 'geometry']

arc = gpd.read_file(RawData_Folder + 'ArcStorm\PARCEL_ARC.gdb',
                    layer          = 'PARCEL_ARC',
                    include_fields = arc_cols)

del [RawData_Folder, arc_cols]

print('         input 1/2 read')

<h1 style="text-align:right;font-size:125%">
נתוני חלקות (לאחר חישוב שדה מספר מזהה ואיחוד מרחבי)

In [ ]:
subprocess.call([r'M:\ncdbScripts\scripts\arcpyPro\PARCEL_ALL_Identifier.bat']);

In [ ]:
subprocess.call([r'M:\ncdbScripts\scripts\arcpyPro\Fronts-Spatial_Join.bat']);

In [ ]:
parcels_cols = ['PARCEL_SDE_', 'Identifier']

parcels = gpd.read_file(Results_Gdb,
                        layer           = 'ParcelAll_x_Polygon_sjoin',
                        include_fields  = parcels_cols,
                        ignore_geometry = True)

parcels = parcels.astype(int)

del [Results_Gdb, parcels_cols]

print('         input 2/2 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️
<h1 style="text-align:right; font-size:125%">
<h1 style="text-align:right; font-size:125%">
:עיבוד נתוני חזיתות
<h1 style="text-align:right; font-size:100%">    
תיקון ערכים שגויים בשדה רדיוס
<h1 style="text-align:right; font-size:100%">
מילוי ערכים חסרים ברדיוס עם 0
<h1 style="text-align:right; font-size:100%">
הגדרת שדה כמספר צף עם שני נקודות עשרוניות
<h1 style="text-align:right; font-size:100%"> 
תיקון סוגי שדות
<h1 style="text-align:right; font-size:100%">
הגדרת שדה של סוג הקו
<h1 style="text-align:right; font-size:100%">
הגדרת שדה לאורך רשום

In [ ]:
print('  9.b Cleaning data')

In [ ]:
arc['RADIUS'].replace(['', ' ',' 0.000 0', ' 0.000 1', ' 0.000 2', ' 0.000 3'], 0.00, inplace=True)
arc['RADIUS'].fillna(0.0, inplace = True)

In [ ]:
arc = arc.astype({'LPOLY_'       : int,
                  'RPOLY_'       : int,
                  'RADIUS'       : float,
                  'FNODE_'       : int,
                  'TNODE_'       : int,
                  'Shape_Length' : float})

In [ ]:
arc['LineType'] = np.where(arc['RADIUS'] == 0, 1, 2)

In [ ]:
#condition = arc[
#                ((arc['Shape_Length'] < 50.0) & (abs(arc['Shape_Length'] - arc['MLENGTH']) > 0.06))
#                                                 |
#                 ((arc['Shape_Length'] >= 50.0) & (abs(arc['Shape_Length'] - arc['MLENGTH']) > 0.1))
#                ]

#arc['LegalLength'] = np.where(condition,  np.nan, arc['MLENGTH'].round(decimals = 2))



arc['LegalLength'] = arc['Shape_Length'].round(decimals=2)

arc.drop(columns=['Shape_Length'], inplace=True)

In [ ]:
print('  9.c Data conversion')

<h1 style="text-align:right; font-size:125%">
:מיזוג נתונים
<h1 style="text-align:right; font-size:100%">    
מיזוג פעם ראשונה לקבלת שדה המתאר חלקה שמאלית

In [ ]:
merge = pd.merge(arc, parcels, how = 'left', left_on = 'LPOLY_', right_on = 'PARCEL_SDE_')

merge.rename(columns={'Identifier' : 'LeftParcelUniqueID'}, inplace=True)

merge.drop(columns='LPOLY_', inplace=True)

merge['LeftParcelUniqueID'] = merge['LeftParcelUniqueID'].fillna(0)
merge['LeftParcelUniqueID'] = merge['LeftParcelUniqueID'].astype(np.int32)

merge['PARCEL_SDE_'] = merge['PARCEL_SDE_'].fillna(0)
merge['PARCEL_SDE_'] = merge['PARCEL_SDE_'].astype(np.int32)


del arc

In [ ]:
merge.reset_index(inplace=True, drop=True)
merge.index = merge.index.astype('Int32')

<h1 style="text-align:right; font-size:100%">    
מיזוג פעם שניה לקבלת שדה המתאר חלקה ימנית

In [ ]:
merge = pd.merge(merge, parcels, how = 'left', left_on = 'RPOLY_', right_on = 'PARCEL_SDE_')

merge.rename(columns = {'Identifier' : 'RightParcelUniqueID', 'FNODE_':'StartPointUniqueID', 'TNODE_':'EndPointUniqueID'}, inplace = True)

merge.drop(columns = ['RPOLY_','PARCEL_SDE__x' ,'PARCEL_SDE__y'], inplace = True)

merge['RightParcelUniqueID'] = merge['RightParcelUniqueID'].fillna(0)
merge['RightParcelUniqueID'] = merge['RightParcelUniqueID'].astype(np.int32)

del parcels

<h1 style="text-align:right; font-size:100%">    
הגדרת שדה למספר זיהוי החזית

In [ ]:
merge['FrontUniqueID'] = range(1, len(merge) + 1)
merge['FrontUniqueID'] = merge['FrontUniqueID'].astype(np.int32)
merge.sort_values(['FrontUniqueID','LeftParcelUniqueID','RightParcelUniqueID'], inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  9.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("FrontUniqueID", int),
                        ("LeftParcelUniqueID", int),
                        ("RightParcelUniqueID", int),
                        ("StartPointUniqueID", int), 
                        ("EndPointUniqueID", int),
                        ("LegalLength", float),
                        ("LineType", int),
                        ("Radius", float)
                        ])

Fronts = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                          geometry = None,
                          crs = 2039)

<h1 style="text-align:right; font-size:100%">
איכלוס שדות

In [ ]:
Fronts['FrontUniqueID'] = merge['FrontUniqueID'].astype(np.int32)

merge['LeftParcelUniqueID'].fillna(0, inplace=True)
Fronts['RightParcelUniqueID'] = merge['LeftParcelUniqueID'] + 5000000
Fronts['LeftParcelUniqueID'] = merge['LeftParcelUniqueID'].astype(np.int32)

merge['RightParcelUniqueID'].fillna(0, inplace=True)
Fronts['RightParcelUniqueID'] = merge['RightParcelUniqueID'] + 5000000
Fronts['RightParcelUniqueID'] = Fronts['RightParcelUniqueID'].astype(np.int32)

Fronts['StartPointUniqueID'] = merge['StartPointUniqueID'].astype(np.int32)

Fronts['EndPointUniqueID'] = merge['EndPointUniqueID'].astype(np.int32)

merge['LegalLength'].fillna(0.0, inplace=True)
Fronts['LegalLength'] = merge['LegalLength'].astype(np.float32)

Fronts['LineType'] = merge['LineType'].astype(np.int32)

Fronts['Radius'] = merge['RADIUS'].astype(np.float)

Fronts['geometry'] = merge['geometry']

In [ ]:
del [merge, new_columns]

<h1 style="text-align:right; font-size:100%">    
תיקון רשת קואורדינטות לרשת ישראל התקפה

In [ ]:
Fronts.to_crs(2039, inplace = True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  9.e Exporting results to product folder')

In [ ]:
#csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\Fronts.csv'
json_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\Fronts.geojson'


#Fronts.drop(columns='geometry').to_csv(csv_dir ,encoding='utf8', index=False)
Fronts.to_file(json_dir, driver = 'GeoJSON')

del [Fronts, json_dir]

In [ ]:
subprocess.call([r'M:\ncdbScripts\scripts\arcpyPro\Fronts-Save as Feature Class.bat']);